In [45]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

from typing_extensions import override

# API 키 정보 로드
load_dotenv(override=True)
print("🔍 PROJECT:", os.getenv("GOOGLE_CLOUD_PROJECT"))
print("🔍 CREDENTIAL FILE:", os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))
# ✅ 추가: vertexai 초기화 (프로젝트 강제 적용)

🔍 PROJECT: knu-ema
🔍 CREDENTIAL FILE: C:/Users/SAMSUNG/AppData/Roaming/gcloud/knu-ema-af3cd6fa4532.json


In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader

# 1. PDF 파일 로드
loader = PyPDFLoader("James Stewart - Calculus, Early Transcendentals, International Metric Edition-CENGAGE Learning (2016).pdf")

# 2. 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)

# 3. 문서 로드 및 분할
docs = loader.load()
split_docs = text_splitter.split_documents(docs)


In [47]:
# 4. 배치 임베딩용 래퍼 클래스 정의
import time
from typing import List, Sequence
from langchain_google_genai import GoogleGenerativeAIEmbeddings

class BatchedEmbeddings:
    def __init__(self, base: GoogleGenerativeAIEmbeddings, batch_size: int = 16):
        self.base = base
        self.batch_size = batch_size

    def __call__(self, text_or_texts):
        # 인풋이 str 이면 embed_query, list 면 embed_documents 로 분기
        if isinstance(text_or_texts, str):
            return self.embed_query(text_or_texts)
        return self.embed_documents(text_or_texts)

    def embed_documents(self, texts: Sequence[str]) -> List[List[float]]:
        all_embeddings: List[List[float]] = []
        for i in range(0, len(texts), self.batch_size):
            batch = texts[i : i + self.batch_size]
            embs = self.base.embed_documents(batch)
            all_embeddings.extend(embs)
            if i + self.batch_size < len(texts):
                print("sleep ", i)
                time.sleep(32)

        return all_embeddings

    def embed_query(self, text: str) -> List[float]:
        # Retriever 쿼리용에도 배치가 필요 없으니 바로 위임
        return self.base.embed_query(text)


In [12]:
# 5. Batched Embeddings 인스턴스 생성
base_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-exp-03-07"
)
embeddings = BatchedEmbeddings(
    base=base_embeddings,
    batch_size=16  # 필요에 따라 조절하세요
)

# 6. FAISS VectorStore 생성 (내부에서 batch 단위로 임베딩 처리)
vectorstore = FAISS.from_documents(split_docs, embeddings)

# 7. Retriever 생성 및 확인
calculus_retriever = vectorstore.as_retriever()
print(f"총 청크 수: {len(split_docs)}")
print(f"첫 청크 미리보기:\n{split_docs[0].page_content[:300]}")

sleep  0
sleep  16
sleep  32
sleep  48
sleep  64
sleep  80
sleep  96
sleep  112
sleep  128
sleep  144
sleep  160
sleep  176
sleep  192
sleep  208
sleep  224
sleep  240
sleep  256
sleep  272
sleep  288
sleep  304
sleep  320
sleep  336
sleep  352
sleep  368
sleep  384
sleep  400
sleep  416
sleep  432
sleep  448
sleep  464
sleep  480
sleep  496
sleep  512
sleep  528
sleep  544
sleep  560
sleep  576
sleep  592
sleep  608
sleep  624
sleep  640
sleep  656
sleep  672
sleep  688
sleep  704
sleep  720
sleep  736
sleep  752
sleep  768
sleep  784
sleep  800
sleep  816
sleep  832
sleep  848
sleep  864
sleep  880
sleep  896
sleep  912
sleep  928
sleep  944
sleep  960
sleep  976
sleep  992
sleep  1008
sleep  1024
sleep  1040
sleep  1056
sleep  1072
sleep  1088
sleep  1104
sleep  1120
sleep  1136
sleep  1152
sleep  1168
sleep  1184
sleep  1200
sleep  1216
sleep  1232
sleep  1248
sleep  1264
sleep  1280
sleep  1296
sleep  1312
sleep  1328
sleep  1344
sleep  1360
sleep  1376
sleep  1392
sleep  1408
sle

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


총 청크 수: 5028
첫 청크 미리보기:
calculus
Early Transc EndEnTals
Eigh Th EdiTion
mETric v Ersion
Jam Es sTE war T
McMaster University  
and  
University of toronto
Australia • Brazil • Mexico • Singapore • United Kingdom • United States
Copyright 2016 Cengage Learning. All Rights Reserved. May not be copied, scanned, or duplicated,


In [ ]:
vectorstore.save_local("vectorstore")

In [64]:
# 5. Batched Embeddings 인스턴스 생성
base_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-exp-03-07"
)

# 6. FAISS VectorStore 생성 (내부에서 batch 단위로 임베딩 처리)
vectorstore = FAISS.load_local("vectorstore", base_embeddings, allow_dangerous_deserialization=True)

# 7. Retriever 생성 및 확인
calculus_retriever = vectorstore.as_retriever()

In [18]:
calculus_retriever.invoke("WHAT IS INTEGRAL")

[Document(id='8f2d5f76-c7a3-4e4b-b3d4-509cf0add101', metadata={'producer': 'Pdftools SDK', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-05-01T06:05:37+00:00', 'source': 'James Stewart - Calculus, Early Transcendentals, International Metric Edition-CENGAGE Learning (2016).pdf', 'total_pages': 1421, 'page': 409, 'page_label': '410'}, page_content='we divide the interval fa, bg into n subintervals of equal width Dx−sb2adyn. \nWe\xa0let x0 s− ad, x1, x2, . . . , x n s−bd be the endpoints of these subintervals and we \nlet x1*, x2*, . . . , x n* be any sample points in these subintervals, so xi* lies in the ith \nsubinterval fxi21, xig. Then the definite integral of f from a to b is\nyb\na fsxddx−lim\nn l ` on\ni−1 fsxi*dDx\n provided that this limit exists and gives the same value for all possible choices of sample points. If it does exist, we say that \nf is integrable on fa, bg.\nThe precise meaning of the limit that defines the integral is as follows:\nFor every number « . 0

In [48]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

# 1) 전체 챕터 MD 파일을 로드
loader = DirectoryLoader(
    path="chapters_md", glob="*.md", loader_cls=TextLoader
)
md_docs = loader.load()

len(md_docs)
md_docs[0]

Document(metadata={'source': 'chapters_md\\Chapter 01 Functions and Models.md'}, page_content='# Chapter 01 Functions and Models\n\n# 함수와 역함수\n\n## 함수의 변환\n\n다음은 함수 그래프의 변환에 대한 설명입니다.  😀\n\n* y축 대칭: $$y = f(-x)$$\n* x축 대칭: $$y = -f(x)$$\n* y축 방향으로 2배 확대: $$y = 2f(x)$$\n* y축 방향으로 1/2배 축소: $$y = \\frac{1}{2}f(x)$$\n* x축 방향으로 2배 확대: $$y = f(\\frac{1}{2}x)$$\n* x축 방향으로 1/2배 축소: $$y = f(2x)$$\n\n\n## 일대일 함수와 역함수\n\n### 일대일 함수\n\n* **정의:** 함수 f가 일대일 함수라는 것은 서로 다른 입력값에 대해 항상 서로 다른 출력값을 갖는다는 것을 의미합니다.  즉, $$x_1 \\ne x_2$$ 이면 $$f(x_1) \\ne f(x_2)$$ 입니다. 🤔\n* **수평선 검정:** 그래프에서 함수가 일대일 함수인지 확인하는 방법은 수평선 검정을 사용하는 것입니다.  어떤 수평선도 그래프와 두 점 이상에서 교차하지 않으면 해당 함수는 일대일 함수입니다.\n\n\n### 역함수\n\n* **정의:**  일대일 함수 $$f$$의 정의역이 A이고 치역이 B일 때, 역함수 $$f^{-1}$$는 정의역이 B이고 치역이 A이며, 모든 $$y \\in B$$에 대해 $$f^{-1}(y) = x \\Leftrightarrow f(x) = y$$ 로 정의됩니다. 🔄\n* **그래프:** $$f^{-1}$$의 그래프는 $$f$$의 그래프를 직선 $$y=x$$에 대해 대칭시켜 얻을 수 있습니다.\n\n\n## 역삼각함수\n\n### 역사인 함수\n\n* **정의:** 역사인 함수 $$f(x) = \\sin^{-1}x$$ 는 $$\\sin y = x$$ 이고 $$

In [49]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document


# 2) 헤더 단위 Splitter
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#",   "Header 1"),   # 대제목
        ("##",  "Header 2"),   # 소제목
        ("###", "Header 3"),   # 소소제목
    ],
    strip_headers=False    # 청크에 헤더 문구를 그대로 남길지 여부
)

chunk_size = 500  # 분할된 청크의 크기를 지정합니다.
chunk_overlap = 100  # 분할된 청크 간의 중복되는 문자 수를 지정합니다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

md_docs_split: list[Document] = []

In [50]:
for doc in md_docs:
    # A) 헤더 단위로 split_text → List[Document]
    header_chunks = markdown_splitter.split_text(doc.page_content)
    for hchunk in header_chunks:
        # hchunk.page_content, hchunk.metadata 에 헤더 정보가 담겨 있음

        # B) 해당 청크를 다시 문자 단위로 자르기 → List[str]
        sub_texts = text_splitter.split_text(hchunk.page_content)
        for sub in sub_texts:
            # C) 메타데이터 합치기
            meta = {**doc.metadata, **hchunk.metadata}

            # (선택) URL 매핑: "Chapter 05 Integrals.md" → "05"
            fname    = doc.metadata["source"].split("/")[-1]
            chap_num = fname.split(" ", 2)[1].zfill(2)
            meta["url"] = f"https://ema.com/theory/{chap_num}"

            # D) 최종 Document 생성
            md_docs_split.append(
                Document(page_content=sub, metadata=meta)
            )


In [58]:
# 검증용 출력 코드

# 전체 청크 개수 출력
print(f"총 분할된 청크 수: {len(md_docs_split)}\n")

# 상위 10개 청크만 미리 보기
for idx, doc in enumerate(md_docs_split[:10], start=1):
    # 메타데이터에서 URL과 헤더 정보만 추출
    url = doc.metadata.get("url", "N/A")
    headers = {k: v for k, v in doc.metadata.items() if k.startswith("Header")}
    preview = doc.page_content.replace("\n", " ")[:200] + "..."

    print(f"--- Chunk #{idx} ---")
    print(f"URL       : {url}")
    print(f"Headers   : {headers}")
    print(f"Preview   : {preview}")
    print()


총 분할된 청크 수: 340

--- Chunk #1 ---
URL       : https://ema.com/theory/01
Headers   : {'Header 1': 'Chapter 01 Functions and Models'}
Preview   : # Chapter 01 Functions and Models...

--- Chunk #2 ---
URL       : https://ema.com/theory/01
Headers   : {'Header 1': '함수와 역함수', 'Header 2': '함수의 변환'}
Preview   : # 함수와 역함수   ## 함수의 변환   다음은 함수 그래프의 변환에 대한 설명입니다.  😀   * y축 대칭: $$y = f(-x)$$ * x축 대칭: $$y = -f(x)$$ * y축 방향으로 2배 확대: $$y = 2f(x)$$ * y축 방향으로 1/2배 축소: $$y = \frac{1}{2}f(x)$$ * x축 방향으로 2배 확대: $$y = f(...

--- Chunk #3 ---
URL       : https://ema.com/theory/01
Headers   : {'Header 1': '함수와 역함수', 'Header 2': '일대일 함수와 역함수', 'Header 3': '일대일 함수'}
Preview   : ## 일대일 함수와 역함수   ### 일대일 함수   * **정의:** 함수 f가 일대일 함수라는 것은 서로 다른 입력값에 대해 항상 서로 다른 출력값을 갖는다는 것을 의미합니다.  즉, $$x_1 \ne x_2$$ 이면 $$f(x_1) \ne f(x_2)$$ 입니다. 🤔 * **수평선 검정:** 그래프에서 함수가 일대일 함수인지 확인하는 방법은 수평선 검...

--- Chunk #4 ---
URL       : https://ema.com/theory/01
Headers   : {'Header 1': '함수와 역함수', 'Header 2': '일대일 함수와 역함수', 'Header 3': '

In [59]:
base_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-exp-03-07"
)
md_embedding = BatchedEmbeddings(
    base=base_embeddings,
    batch_size=16  # 필요에 따라 조절하세요
)

# 6. FAISS VectorStore 생성 (내부에서 batch 단위로 임베딩 처리)
md_vectorstore = FAISS.from_documents(md_docs_split, md_embedding)

# 7. Retriever 생성 및 확인
md_retriever = md_vectorstore.as_retriever()
print(f"총 청크 수: {len(md_docs)}")
print(f"첫 청크 미리보기:\n{md_docs[0].page_content[:300]}")

sleep  0
sleep  16
sleep  32
sleep  48
sleep  64
sleep  80
sleep  96
sleep  112
sleep  128
sleep  144
sleep  160
sleep  176
sleep  192
sleep  208
sleep  224
sleep  240
sleep  256
sleep  272
sleep  288
sleep  304
sleep  320


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


총 청크 수: 15
첫 청크 미리보기:
# Chapter 01 Functions and Models

# 함수와 역함수

## 함수의 변환

다음은 함수 그래프의 변환에 대한 설명입니다.  😀

* y축 대칭: $$y = f(-x)$$
* x축 대칭: $$y = -f(x)$$
* y축 방향으로 2배 확대: $$y = 2f(x)$$
* y축 방향으로 1/2배 축소: $$y = \frac{1}{2}f(x)$$
* x축 방향으로 2배 확대: $$y = f(\frac{1}{2}x)$$
* x축 방향으로 1/2배 축소: $$y = f(2x)$$


## 일대일 함수와 역함수

#


In [60]:
md_vectorstore.save_local("md_vectorstore")

In [68]:
# 6. FAISS VectorStore 생성 (내부에서 batch 단위로 임베딩 처리)
md_vectorstore = FAISS.load_local("md_vectorstore", base_embeddings, allow_dangerous_deserialization=True)

# 7. Retriever 생성 및 확인
md_retriever = md_vectorstore.as_retriever()

result = md_retriever.invoke("정적분이 뭐야?")
print(result[0].metadata["url"])

https://ema.com/theory/04


In [74]:
from langchain.tools import Tool
def calculus_search_fn(query: str) -> list[dict]:
    docs = calculus_retriever.get_relevant_documents(query)
    return [
        {
            "text": doc.page_content,
            "chapter": doc.metadata.get("Header 1"),
            "section": doc.metadata.get("Header 2")
        }
        for doc in docs
    ]

calculus_tool = Tool.from_function(
    calculus_search_fn,
    name="calculus_search",
    description = (
        "Use this tool for semantic retrieval over the Calculus textbook, "
        "optimized for scholarly and academic-level inquiries. Given a user’s "
        "natural-language question about any calculus concept (e.g., definitions, "
        "theorems, proofs, examples, formulas, applications), return the most "
        "relevant textbook sections and excerpts along with metadata (chapter and "
        "section headings) so that the agent can provide accurate, "
        "context-aware, and authoritative answers."
    )
)
# 이 도구는 미적분 교재 전체를 대상으로 학술적 수준의 심도 있는 질문에 최적화된 시맨틱 검색을 수행합니다.
# 사용자가 미적분 개념(예: 정의, 정리, 증명, 예제, 공식, 응용 등)에 대해 자연어로 묻는 경우,
# 챕터·단원 제목과 URL 등의 메타데이터와 함께 가장 관련성 높은 교재 발췌를 반환하여
# 에이전트가 정확하고 맥락에 맞으며 권위 있는 답변을 제공할 수 있도록 지원합니다.

def md_search_fn(query: str) -> list[dict]:
    docs = md_retriever.get_relevant_documents(query)
    return [
        {
            "text": doc.page_content,
            "chapter": doc.metadata.get("Header 1"),
            "section": doc.metadata.get("Header 2"),
            "url": doc.metadata["url"],
        }
        for doc in docs
    ]

md_tool = Tool.from_function(
    md_search_fn,
    name="md_search",
    description=(
        "Use this tool to semantically search the user-friendly, markdown-formatted study guide for calculus. "
        "Given a natural-language question about any calculus concept, it returns the most relevant markdown summary "
        "sections along with their static page URLs, enabling the agent to suggest “please check this page for more details.”"
    )
)
# 이 도구는 사용자 친화적인 Markdown 형식의 미적분 학습 가이드를 시맨틱 검색하는 데 사용됩니다.
# 학술 수준의 질문과 보다 심도 있는 개념 이해를 지원하도록 설계되었으며, 가장 관련성 높은 간결한 설명과
# 해당 정적 페이지 URL을 반환합니다. 에이전트는 “보다 학술적인 개요나 자세한 예시는 이 페이지를 참조하세요”
# 라고 제안할 때 이 도구를 호출하면 됩니다.

In [69]:
from langchain.tools.retriever import create_retriever_tool
calculus_retriever_tool = create_retriever_tool(
    calculus_retriever,
    name = "calculus_search",
    description = (
        "Use this tool for semantic retrieval over the Calculus textbook, "
        "optimized for scholarly and academic-level inquiries. Given a user’s "
        "natural-language question about any calculus concept (e.g., definitions, "
        "theorems, proofs, examples, formulas, applications), return the most "
        "relevant textbook sections and excerpts along with metadata (chapter and "
        "section headings) so that the agent can provide accurate, "
        "context-aware, and authoritative answers."
    )
)
# 이 도구는 미적분 교재 전체를 대상으로 학술적 수준의 심도 있는 질문에 최적화된 시맨틱 검색을 수행합니다.
# 사용자가 미적분 개념(예: 정의, 정리, 증명, 예제, 공식, 응용 등)에 대해 자연어로 묻는 경우,
# 챕터·단원 제목과 URL 등의 메타데이터와 함께 가장 관련성 높은 교재 발췌를 반환하여
# 에이전트가 정확하고 맥락에 맞으며 권위 있는 답변을 제공할 수 있도록 지원합니다.

md_retriever_tool = create_retriever_tool(
    md_retriever,
    name="md_search",
    description=(
        "Use this tool to semantically search the user-friendly, markdown-formatted study guide for calculus. "
        "Given a natural-language question about any calculus concept, it returns the most relevant markdown summary "
        "sections along with their static page URLs, enabling the agent to suggest “please check this page for more details.”"
    )
)
# 이 도구는 사용자 친화적인 Markdown 형식의 미적분 학습 가이드를 시맨틱 검색하는 데 사용됩니다.
# 학술 수준의 질문과 보다 심도 있는 개념 이해를 지원하도록 설계되었으며, 가장 관련성 높은 간결한 설명과
# 해당 정적 페이지 URL을 반환합니다. 에이전트는 “보다 학술적인 개요나 자세한 예시는 이 페이지를 참조하세요”
# 라고 제안할 때 이 도구를 호출하면 됩니다.

In [75]:
tools = [calculus_tool, md_tool]

In [77]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash", temperature=0.2)

# Prompt 정의
theory_explanation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are the ‘Theory Explanation Agent’, specialized in answering and explaining calculus concepts. "
            "When the user asks a theoretical question about calculus, you should:\n"
            "1. Use the `calculus_search` tool to semantically retrieve authoritative passages from the Calculus textbook.\n"
            "2. Use the `md_search` tool to fetch relevant markdown-formatted study guide sections and their static page URLs.\n"
            "3. Please answer in a way that’s easy for Koreans to understand.\n"
            "Always cite the retrieved context in your explanation. If neither tool yields an answer, fall back to your internal knowledge. "
            "Structure your response as a clear, scholarly explanation, and suggest the static page URL when appropriate."
        ),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# system:
# “당신은 ‘이론설명 에이전트’로, 미적분 이론 질문에 답하고 개념을 설명하는 데 특화되어 있습니다.
# 사용자가 미적분 이론에 대해 묻는 경우 다음을 수행하세요:
# 
# calculus_search 도구를 사용해 교재에서 권위 있는 내용을 시맨틱 검색합니다.
# 
# md_search 도구를 사용해 Markdown 형식 학습 가이드의 관련 섹션과 정적 페이지 URL을 제공합니다.
# 검색한 컨텍스트는 설명에 반드시 인용하세요. 도구로 답을 찾지 못하면 내부 지식을 활용합니다.
# 답변은 학술적으로 명확하게 구성하고, 필요 시 정적 페이지 URL을 제안하세요.”

In [78]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, theory_explanation_prompt)


In [79]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [86]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

In [100]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.invoke(
    {"input": "∫xe^xdx 를 부분적분법(Integration by Parts)을 하기가 어렵네. 공부할 수 있는 페이지 있어?"}
)

for i in result:
    print(result[i])


∫xe^xdx 를 부분적분법(Integration by Parts)을 하기가 어렵네. 공부할 수 있는 페이지 있어?
 부분적분법은 곱의 형태로 이루어진 함수의 적분을 계산하는 데 유용한 기법입니다. (https://ema.com/theory/07)

부분적분 공식은 다음과 같습니다:
$\\int u dv = uv - \\int v du$

$\\int xe^x dx$를 계산하기 위해, u와 dv를 선택해야 합니다. 여기서 u는 미분했을 때 더 간단해지는 함수로 선택하는 것이 좋습니다. 이 경우, x를 u로, $e^x dx$를 dv로 선택합니다.

u = x, dv = $e^x dx$
du = dx, v = $e^x$

부분적분 공식에 대입하면:
$\\int xe^x dx = xe^x - \\int e^x dx = xe^x - e^x + C$

따라서, $\\int xe^x dx = xe^x - e^x + C$입니다.

자세한 내용은 다음 페이지를 참조하십시오: https://ema.com/theory/07

